# Filter ATL08

1. Get the BBOX by TileID from 3.x notebooks
2. Query CMR for matching ATL08 granules
3. Get the IDs (later download urls)
4. list the csv (or read from the 

In [119]:
import os
import itertools
import copy

#from notebooks.general.covariateutils import get_index_tile
#import notebooks.general.CovariateUtils 
from CovariateUtils import get_index_tile
# import the MAAP package
from maap.maap import MAAP

# create MAAP class
maap = MAAP()


In [37]:
os.getcwd()

'/projects/icesat2_boreal/notebooks/2.ICESat-2_processing'

In [15]:
#bbox = '-101.5,45.5,-100.5,46.5'

geojson_path_albers = "/projects/maap-users/alexdevseed/boreal_tiles.gpkg"
layer = "boreal_tiles_albers"
tile_n = 30543

tile_id = get_index_tile(geojson_path_albers, tile_n, buffer=0, layer = layer)

in_bbox = ",".join(str(coord) for coord in tile_id['bbox_4326'])
out_crs = tile_id['tile_crs']
in_bbox

'-116.85483935749697,50.57021397455074,-116.26013039328576,50.94612905730388'

In [62]:

# generated from this EDSC search: https://ade.maap-project.org:30052/search?q=ATL08
# TODO pass more filtering
results = maap.searchGranule(short_name="ATL08",
                             version="003", 
                             bounding_box=in_bbox)

In [66]:
type(results[0])

maap.Result.Granule

In [28]:
csv = [os.path.basename(item.getDownloadUrl()) for item in results]
csv

['ATL08_20181020224531_03410102_003_01.h5',
 'ATL08_20181104102812_05620106_003_01.h5',
 'ATL08_20181118212141_07830102_003_01.h5',
 'ATL08_20181122211318_08440102_003_01.h5',
 'ATL08_20181203090405_10040106_003_01.h5',
 'ATL08_20181207085545_10650106_003_01.h5',
 'ATL08_20181221194919_12860102_003_01.h5',
 'ATL08_20190101074013_00590206_003_01.h5',
 'ATL08_20190105073153_01200206_003_01.h5',
 'ATL08_20190119182518_03410202_003_01.h5',
 'ATL08_20190203060805_05620206_003_01.h5',
 'ATL08_20190217170132_07830202_003_01.h5',
 'ATL08_20190221165313_08440202_003_01.h5',
 'ATL08_20190304044405_10040206_003_01.h5',
 'ATL08_20190308043545_10650206_003_01.h5',
 'ATL08_20190322152915_12860202_003_01.h5',
 'ATL08_20190402032005_00590306_003_01.h5',
 'ATL08_20190406031145_01200306_003_01.h5',
 'ATL08_20190420140513_03410302_003_01.h5',
 'ATL08_20190505014742_05620306_003_01.h5']

In [132]:
YEARS = [2019, 2020]
DATE_START = '06-01T00:00:00Z' # SUMMER
DATE_END = '09-30T23:59:59Z' # SUMMER

date_filters = [f'{year}-{DATE_START},{year}-{DATE_END}' for year in YEARS]
date_filters

['2019-06-01T00:00:00Z,2019-09-30T23:59:59Z',
 '2020-06-01T00:00:00Z,2020-09-30T23:59:59Z']

In [136]:
queries2 = [{
    'short_name':"ATL08",
    'version':"003", 
    'bounding_box':in_bbox,
    'temporal':date_filter[0]
    },
    {
    'short_name':"ATL08",
    'version':"003", 
    'bounding_box':in_bbox,
    'temporal':date_filter[1]
    }
]


def build_query(base_query: dict, date: str) -> dict:
    base_query["temporal"] = date
    return base_query

base_query = {
    'short_name':"ATL08",
    'version':"003",
    'bounding_box':in_bbox
}
    
q3 = [build_query(copy.copy(base_query), date_filter) for date_filter in date_filters]
queries = [dict(base_query, temporal=date_filter) for date_filter in date_filters]

In [135]:
q3

[{'short_name': 'ATL08',
  'version': '003',
  'bounding_box': '-116.85483935749697,50.57021397455074,-116.26013039328576,50.94612905730388',
  'temporal': '2019-06-01T00:00:00Z,2019-09-30T23:59:59Z'},
 {'short_name': 'ATL08',
  'version': '003',
  'bounding_box': '-116.85483935749697,50.57021397455074,-116.26013039328576,50.94612905730388',
  'temporal': '2020-06-01T00:00:00Z,2020-09-30T23:59:59Z'}]

In [137]:
# query CMR as many seasons as necessary
result_chain = itertools.chain.from_iterable([maap.searchGranule(**query) for query in queries])

In [138]:
# get the s3 urls for granules we want to process
granules = [item.getDownloadUrl() for item in result_chain]

In [116]:
# now convert each granule to las format, keeping only relevant dimensions
# eventually make this a dps job to be run over hundreds of granules
# granules only ever need to be converted once, unless dimensions change

In [139]:
granules

['s3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/ATL08_20190603002339_10040306_003_01.h5',
 's3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/ATL08_20190607001520_10650306_003_01.h5',
 's3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/ATL08_20190621110851_12860302_003_01.h5',
 's3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/ATL08_20190720094445_03410402_003_01.h5',
 's3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/ATL08_20190803212724_05620406_003_01.h5',
 's3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/ATL08_20190818082057_07830402_003_01.h5',
 's3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/ATL08_20190822081238_08440402_003_01.h5',
 's3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/ATL08_20190901200329_10040406_003_01.h5',
 's3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/ATL08_20190905195508_10650406_003_01.h5',
 's3://nasa-maap-data-store/file-staging/nasa-map/ATL08___003/AT